Set Up

In [ ]:
from pathlib import Path
import pandas as pd
from utils.config import load_config, abspath, ensure_dirs
from utils.preprocessing import run_basic_processing_and_save

cfg, root = load_config()
cfg, root


In [ ]:
df, out_ds = run_basic_processing_and_save(cfg, root)
df.shape, out_ds


In [ ]:
import matplotlib.pyplot as plt

# GHQ caseness
if "target_cls" in df:
    df["target_cls"].value_counts(dropna=False).plot(kind="bar")
    plt.title("GHQ caseness (n_scghq2_dv >= 4)")
    plt.show()

# SF-12 MCS 分布
if "target_reg" in df:
    df["target_reg"].dropna().plot(kind="hist", bins=40)
    plt.title("SF-12 MCS distribution")
    plt.show()


In [ ]:
# 简单缺失率
na = df.isna().mean().sort_values(ascending=False).head(30)
na.to_frame("missing_rate").head(20)
